# Optimization project Iver & Alessandro 

In [9]:
import gurobipy as gp
from gurobipy import GRB
from enum import Enum
import numpy as np

## Taxi notes

Grid with coordinates 1-25

| 01 | 02 | 03 | 04 | 05 |
| 06 | 07 | 08 | 09 | 10 | etc.

Possible states:
Taxi location 1-25
Taxi empty or not

Possible actions:
N (-5), S (+5), E (+1), W (-1)
Pick up, drop off

Rewards:
-1 per time step
+20 for successfull drop off
-10 for trying to drop off when it cant

Initial state: Taxi at pos 1


Goal:

find a policy $\mu$ that for each state $s$ chooses the optimal action $a$ such that

$$
\mu (s, a) = argmax Q(s, a)
$$



## Linear Models

### 1.

maximize -#steps + 20 * successfull_dropoff - 10 * failed_passenger_operation
s.t.


In [10]:
MAP_SIZE = 5

class Tile(Enum):
    EMPTY = 0
    BUILDING = 1
    PASSENGER = 2
    DROP_OFF = 3


# Remember to index as [y, x]
mappy = np.array(
[
    [0, 1, 0, 0, 0,],
    [0, 0, 0, 1, 0,],
    [0, 1, 0, 1, 3,],
    [0, 1, 0, 0, 0,],
    [2, 1, 1, 0, 0,]
])


In [11]:
class Action(Enum):
    MOVE_SOUTH = 1
    MOVE_NORTH = 2
    MOVE_EAST = 3
    MOVE_WEST = 4
    PICK_UP = 5
    DROP_OFF = 6


class FaultyPassengerAction(Exception):
    pass

class FaultyMove(Exception):
    pass

def do_action(x, y, passenger, action) -> tuple[int, int, bool, Exception | None]:
    if action == Action.PICK_UP:
        if passenger:
            return x, y, passenger, FaultyPassengerAction
        return x, y, True, None
    elif action == Action.DROP_OFF:
        if not passenger:
            return x, y, passenger, FaultyPassengerAction
        return x, y, False, None

    nx, ny = x, y
    if action == Action.MOVE_NORTH:
        ny -= 1
    elif action == Action.MOVE_SOUTH:
        ny += 1
    elif action == Action.MOVE_EAST:
        nx += 1
    elif action == Action.MOVE_WEST:
        nx -= 1

    if nx < 0 or ny < 0 or nx >= MAP_SIZE or ny >= MAP_SIZE:
        return x, y, passenger, FaultyMove
    elif mappy[ny, nx] == Tile.BUILDING:
        return x, y, passenger, FaultyMove
    else:
        return nx, ny, passenger, None

def get_idx(x: int, y: int, passenger: int | bool) -> int:
    return x + y * MAP_SIZE + int(passenger) * MAP_SIZE * MAP_SIZE

In [36]:
m = gp.Model("somename")

# Vs[y, x, passenger] => Value of state
Vs = m.addMVar((MAP_SIZE * MAP_SIZE * 2), lb=0)
# choices[y, x, p, ny, nx, np] => probability of transition
identity = np.identity(MAP_SIZE * MAP_SIZE * 2)
elementary_vector = np.ones((MAP_SIZE * MAP_SIZE * 2))
possibilities = np.zeros((MAP_SIZE * MAP_SIZE * 2, MAP_SIZE * MAP_SIZE * 2))
choices = m.addMVar((MAP_SIZE * MAP_SIZE * 2, MAP_SIZE * MAP_SIZE * 2), lb=0, ub=1)
rewards = np.zeros((MAP_SIZE * MAP_SIZE * 2, MAP_SIZE * MAP_SIZE * 2))
gamma = 0.5

m.setObjective(
    (1 - gamma) * gp.quicksum(Vs),
    # * gp.quicksum(
    #     [
    #         Vs[get_idx(x, y, passenger)]
    #         for x in range(MAP_SIZE)
    #         for y in range(MAP_SIZE)
    #         for passenger in [0, 1]
    #         if mappy[y, x] != Tile.BUILDING
    #     ]
    # ),
    sense=GRB.MINIMIZE,
)

for x in range(MAP_SIZE):
    for y in range(MAP_SIZE):
        if mappy[y, x] == Tile.BUILDING:
            # Skip all buildings
            continue  # unsure if this is the correct thing to do
        for passenger in [0, 1]:
            all_Qs = []
            for action in Action:
                # Enforcing V(s) >= R(s,a) + V(s')
                reward = -1  # For using a timestep

                nx, ny, npass, error = do_action(x, y, passenger, action)
                if error == FaultyPassengerAction:
                    reward -= 10
                elif action == Action.DROP_OFF:
                    # Successfull drop off
                    reward += 20
                elif error == FaultyMove:
                    continue  # Skip all buildings
                    # reward = -1_000_000_000

                possibilities[get_idx(x, y, passenger), get_idx(nx, ny, npass)] = 1.0
                rewards[get_idx(x, y, passenger), get_idx(nx, ny, npass)] = reward

for i in range(MAP_SIZE * MAP_SIZE * 2):
    m.addConstr(gp.quicksum([choices[i, j] for j in range(MAP_SIZE * MAP_SIZE * 2)]) == 1.0)
m.addConstr((identity - gamma * (choices * possibilities)) * Vs >= elementary_vector * ((choices * possibilities) * rewards))
m.optimize()

print()
for x in range(MAP_SIZE):
    for y in range(MAP_SIZE):
        if mappy[y, x] == Tile.BUILDING:
            # Skip all buildings
            continue  # unsure if this is the correct thing to do
        print(Vs[get_idx(x, y, True)].X, end=" ")
    print()
        # for passenger in [0, 1]:


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 50 rows, 2550 columns and 2500 nonzeros
Model fingerprint: 0x5592f337
Model has 2500 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [5e-01, 5e-01]
  QLMatrix range   [1e+00, 2e+01]
  Objective range  [5e-01, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 2265 columns


GurobiError: Q matrix is not positive semi-definite (PSD). Set NonConvex parameter to 2 to solve model.